In [1]:

!pip install torch==2.2.2 torchtext==0.6.0
!pip install spacy
!pip install "numpy<2"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 122.0 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
jax 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
shap 0.50.0 requires numpy>=2, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; pytho

In [1]:

import torch
import torchtext
print(f"Torch: {torch.__version__}")
# print(f"TorchText: {torchtext.__version__}")

Torch: 2.2.2+cu121


In [2]:
!python -m spacy download
!python -m spacy download en

⚠ As of spaCy v3.0, shortcuts like 'de' are deprecated. Please use the
full pipeline package name 'de_core_news_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.6/14.6 MB 121.0 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('de_core_news_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.
⚠ As of spaCy v3.0, shortcuts like 'en' are deprecated. Please use the
full pipeline package name 'en_core_web_sm' instead.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 127.6 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependenc

In [3]:
import torch
import torch.nn as nn

class Transformer(nn.Module):
    def __init__(
            self,
            embed_size,
            src_vocab_size,
            trg_vocab_size,
            src_pad_idx,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            forward_expansion,
            dropout,
            max_length,
            device
    ):
        super(Transformer, self).__init__()
        self.src_embedding = nn.Embedding(src_vocab_size, embed_size)
        self.src_positional_embedding = nn.Embedding(max_length, embed_size)
        self.trg_embedding = nn.Embedding(trg_vocab_size, embed_size)
        self.trg_positional_embedding = nn.Embedding(max_length, embed_size)
        self.device = device

        self.transformer = nn.Transformer(
            embed_size,
            num_heads,
            num_encoder_layers,
            num_decoder_layers,
            forward_expansion,
            dropout,
        )
        self.fc_out = nn.Linear(embed_size, trg_vocab_size)
        self.dropout = nn.Dropout(dropout)
        self.src_pad_idx = src_pad_idx

    def make_src_mask(self, src):
        # src   => (src_len, N)
        # mask  => (N, src_len)
        return src.transpose(0, 1) == self.src_pad_idx

    def forward(self, src, trg):
        src_seq_length, N = src.shape
        trg_seq_length, N = trg.shape

        src_postions = (
            torch.arange(0, src_seq_length).unsqueeze(1).expand(src_seq_length, N).to(self.device)
        )

        trg_postions = (
            torch.arange(0, trg_seq_length).unsqueeze(1).expand(trg_seq_length, N).to(self.device)
        )

        embed_src = self.dropout(
            self.src_embedding(src) + self.src_positional_embedding(src_postions)
        )

        embed_trg = self.dropout(
            self.src_embedding(trg) + self.src_positional_embedding(trg_postions)
        )

        src_padding_mask = self.make_src_mask(src)
        trg_padding_mask = self.transformer.generate_square_subsequent_mask(trg_seq_length).to(self.device)

        out = self.transformer(
            embed_src,
            embed_trg,
            src_key_padding_mask=src_padding_mask,
            tgt_mask=trg_padding_mask
        )

        return self.fc_out(out)


## Decoding Function

In [4]:
import torch
import spacy
from torchtext.data.metrics import bleu_score
import sys


def decoding(model, sentence, german, english, device, max_length=50):
    # Load german tokenizer
    spacy_ger = spacy.load("de_core_news_sm")

    # Create tokens using spacy and everything in lower case (which is what our vocab is)
    if type(sentence) == str:
        tokens = [token.text.lower() for token in spacy_ger(sentence)]
    else:
        tokens = [token.lower() for token in sentence]

    # Add <SOS> and <EOS> in beginning and end respectively
    tokens.insert(0, german.init_token)
    tokens.append(german.eos_token)

    # Go through each german token and convert to an index
    text_to_indices = [german.vocab.stoi[token] for token in tokens]

    # Convert to Tensor
    sentence_tensor = torch.LongTensor(text_to_indices).unsqueeze(1).to(device)

    outputs = [english.vocab.stoi["<sos>"]]
    for i in range(max_length):
        trg_tensor = torch.LongTensor(outputs).unsqueeze(1).to(device)

        with torch.no_grad():
            output = model(sentence_tensor, trg_tensor)

        best_guess = output.argmax(2)[-1, :].item()
        outputs.append(best_guess)

        if best_guess == english.vocab.stoi["<eos>"]:
            break

    translated_sentence = [english.vocab.itos[idx] for idx in outputs]
    # remove start token
    return translated_sentence[1:]

In [22]:
def bleu(data, model, german, english, device):
    targets = []
    outputs = []

    for example in data:
        src = vars(example)["src"]
        trg = vars(example)["trg"]
        print(src)
        print(trg)
        prediction = decoding(model, src, german, english, device)
        prediction = prediction[:-1]  # remove <eos> token

        targets.append([trg])
        outputs.append(prediction)
    print(outputs)
    print(targets)

    return bleu_score(outputs, targets)


def save_checkpoint(state, filename="my_checkpoint.pth.tar"):
    print("=> Saving checkpoint")
    torch.save(state, filename)


def load_checkpoint(checkpoint, model, optimizer):
    print("=> Loading checkpoint")
    model.load_state_dict(checkpoint["state_dict"])
    optimizer.load_state_dict(checkpoint["optimizer"])

In [14]:
import os
import requests

# 1. Create the directory torchtext expects
os.makedirs('.data/multi30k', exist_ok=True)

# 2. Define the URLs for the dataset (using a reliable mirror)
urls = [
    ('https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/train.de', 'train.de'),
    ('https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/train.en', 'train.en'),
    ('https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/val.de', 'val.de'),
    ('https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/val.en', 'val.en'),
    ('https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/test2016.de', 'test2016.de'),
    ('https://raw.githubusercontent.com/neychev/small_DL_repo/master/datasets/Multi30k/test2016.en', 'test2016.en')
]

# 3. Download the files
print("Downloading Multi30k dataset manually...")
for url, filename in urls:
    response = requests.get(url)
    with open(f'.data/multi30k/{filename}', 'wb') as f:
        f.write(response.content)
    print(f"Downloaded {filename}")

print("Download complete. You can now run the Multi30k.splits command.")

Downloaded train.de
Downloaded train.en
Downloaded val.de
Downloaded val.en
Downloaded test2016.de
Downloaded test2016.en
Download complete. You can now run the Multi30k.splits command.


In [28]:
from datasets import load_dataset

ds = load_dataset("bentrevett/multi30k")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

train.jsonl: 0.00B [00:00, ?B/s]

val.jsonl: 0.00B [00:00, ?B/s]

test.jsonl: 0.00B [00:00, ?B/s]

Generating train split:   0%|          | 0/29000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/1014 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/1000 [00:00<?, ? examples/s]

## Tokenization

In [24]:
import spacy
from torch.utils.tensorboard import SummaryWriter
from torchtext.datasets import Multi30k
from torchtext import data

spacy_ger = spacy.load('de_core_news_sm')
spacy_eng = spacy.load('en_core_web_sm')

def tokenize_ger(text):
    return [token.text for token in spacy_ger.tokenizer(text)]

def tokenize_eng(text):
    return [token.text for token in spacy_eng.tokenizer(text)]

german = data.Field(tokenize=tokenize_ger, lower=True, init_token="<sos>", eos_token="<eos>")
english = data.Field(tokenize=tokenize_eng, lower=True, init_token="<sos>", eos_token="<eos>")

train_data, valid_data, test_data = Multi30k.splits(
    exts=(".de", ".en"), fields=(german, english)
)

german.build_vocab(train_data, max_size=10000, min_freq=2)
english.build_vocab(train_data, max_size=10000, min_freq=2)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
load_model = False
save_model = True


## Hyperparameters

In [16]:
# Training hyperparameters
num_epochs = 1
lr = 3e-4
bs = 32

# Model hyperparameters
src_vocab_size      = len(german.vocab)
trg_vocab_size      = len(english.vocab)
embed_size          = 512
num_heads           = 8
num_encoder_layers  = 3
num_decoder_layers  = 3
dropout             = 0.1
max_length          = 100
forward_expansion   = 4
src_pad_idx         = english.vocab.stoi["<pad>"]

## Tensorboard

In [17]:
writer = SummaryWriter("runs/loss_plot")
step = 0

## Data split

In [18]:
train_iterator, valid_iterator, test_iterator = data.BucketIterator.splits(
    (train_data, valid_data, test_data),
    batch_size = bs,
    sort_within_batch = True,
    sort_key = lambda x: len(x.src),
    device = device
)

In [27]:
for batch in train_iterator:
  print(batch)


[torchtext.data.batch.Batch of size 1 from MULTI30K]
	[.src]:[torch.cuda.LongTensor of size 6x1 (GPU 0)]
	[.trg]:[torch.cuda.LongTensor of size 6x1 (GPU 0)]


In [23]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.nn.utils import clip_grad_norm

model = Transformer(
    embed_size,
    src_vocab_size,
    trg_vocab_size,
    src_pad_idx,
    num_heads,
    num_encoder_layers,
    num_decoder_layers,
    forward_expansion,
    dropout,
    max_length,
    device
).to(device)

optimizer = optim.Adam(model.parameters(), lr=lr)

criterion = nn.CrossEntropyLoss(ignore_index=src_pad_idx)

if load_model:
    load_checkpoint(torch.load("my_checkpoint.pth.ptar"), model, optimizer)

for epoch in range(num_epochs):
    print(f"[Epoch {epoch} / {num_epochs}]")

    # if save_model:
    #     checkpoint = {
    #         "state_dict" : model.state_dict(),
    #         "optimizer" : optimizer.state_dict()
    #     }

    #     save_checkpoint(checkpoint)

    model.train()

    for batch_idx, batch in enumerate(train_iterator):
        inp_data = batch.src.to(device)
        target = batch.trg.to(device)
        output = model(inp_data, target[:-1])

        output = output.reshape(-1, output.shape[2])
        target = target[1:].reshape(-1)
        optimizer.zero_grad()

        loss = criterion(output, target)
        loss.backward()
        torch.nn.utils.clip_grad_norm(model.parameters(), max_norm=1)
        # print("==="*50)
        # print(inp_data)
        # print(target)
        # print(output[-1])
        # print(loss.item())
        # print("==="*50)

        optimizer.step()
        writer.add_scalar("Training loss", loss, global_step=step)
        step += 1

score = bleu(test_data, model, german, english, device)
print(f"Bleu score {score*100:.2f}")

sentence =  "ein pferd geht unter einer brücke neben einem boot."

translated_sentence = decoding(
    model, sentence, german, english, device, max_length = max_length
)

print(f"Translated example sentence \n {translated_sentence}")

/usr/local/lib/python3.12/dist-packages/torch/nn/modules/transformer.py:286: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")
/tmp/ipython-input-357918164.py:51: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.
  torch.nn.utils.clip_grad_norm(model.parameters(), max_norm=1)


[Epoch 0 / 1]
['404', ':', 'not', 'found']
['404', ':', 'not', 'found']
[['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>']]
[[['404', ':', 'not', 'found']]]
Bleu score 0.00
Translated example sentence 
 ['<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>', '<unk>',

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.0/18.0 MB 117.4 MB/s eta 0:00:00
  Attempting uninstall: numpy
    Found existing installation: numpy 2.0.2
    Uninstalling numpy-2.0.2:
      Successfully uninstalled numpy-2.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
jaxlib 0.7.2 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python-headless 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
pytensor 2.35.1 requires numpy>=2.0, but you have numpy 1.26.4 which is incompatible.
opencv-python 4.12.0.88 requires numpy<2.3.0,>=2; python_version >= "3.9", but you have numpy 1.26.4 which is incompatible.
torchvision 0.24.0+cu126 requires torch==2.9.0, but you have torch 2.2.2 which is incompati